In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

np.random.seed(42)

from collections import deque
from datetime import datetime

from ranked_probability_score import ranked_probability_score

In [8]:
team_mapping = {
    'Bournemouth': 'AFC Bournemouth',
    'Brighton': 'Brighton and Hove Albion',
    'Huddersfield': 'Huddersfield Town',
    'Leeds': 'Leeds United',
    'Leicester': 'Leicester City',
    'Manchester Utd': 'Manchester United',
    'Newcastle Utd': 'Newcastle',
    'Tottenham': 'Tottenham Hotspur',
    'Sheffield Utd': 'Sheffield United',
    'West Brom': 'West Bromwich Albion',
    'West Ham': 'West Ham United',
    'Wolves': 'Wolverhampton'
}

In [12]:
spi = pd.read_csv('../../data/fivethirtyeight/spi_matches.csv')
spi = spi.loc[(spi['league_id'] == 2411) & (spi['league_id'] == 2411)]
spi = spi.loc[(spi['season'] > 2016)]
spi.columns

Index(['season', 'date', 'league_id', 'league', 'team1', 'team2', 'spi1',
       'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2'],
      dtype='object')

In [19]:
X = spi[['team1', 'team2']]

y = np.select(
        [
            spi["score1"] > spi["score2"],
            spi["score1"] == spi["score2"],
            spi["score1"] < spi["score2"],
        ],
        [
            0,
            1,
            2,
        ],
        default=1,
    )

In [20]:
X_train = X.loc[spi.season != 2021]
X_test = X.loc[spi.season == 2021]

y_train = y[spi.season != 2021]
y_test = y[spi.season == 2021]

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', SVC())])

pipe.fit(X_train, y_train)

pipe.score(X_test, y_test)

ValueError: could not convert string to float: 'Arsenal'

In [ ]:
y_hat = pipe.predict(X_test)

rps = []

for idx in range(len(y_hat)):
    pred = y_hat[idx]
    truth = y_test[idx]

    pdf = np.zeros(3)
    pdf[pred] = 1
    rps.append(ranked_probability_score(pdf, truth))

np.mean(rps)